In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from src.datamodule.av2_dataset import Av2Dataset
from src.datamodule.av2_extractor import Av2Extractor
from src.datamodule.av2_dataset import collate_fn
from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap
from src.utils.vis import  visualize_scenario
from src.model.trainer_forecast import Trainer as Model

data_root = Path("/home/jcheng/dataset/av2/val") # Path to the datase
ckpt = "./checkpoints/model_forecast_finetune.ckpt"
dataset = Av2Dataset(data_root=data_root, extractor=Av2Extractor(radius=150))
model = Model.load_from_checkpoint(ckpt, pretrain_weights=None)
model = model.eval()

In [ ]:
data = dataset[16]
scene_id = data["scenario_id"]
scene_file = data_root / scene_id / ("scenario_" + scene_id + ".parquet")
map_file = data_root / scene_id / ("log_map_archive_" + scene_id + ".json")
scenario = scenario_serialization.load_argoverse_scenario_parquet(scene_file)
static_map = ArgoverseStaticMap.from_json(map_file)

prediction, prob = model.predict(collate_fn([data]))
visualize_scenario(scenario, static_map, prediction=prediction.squeeze(0))